Análise exploratória dos dados da ONS de Janeiro/2024, disponíveis em dados.ons.org.br

In [140]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import locale
import plotly.graph_objects as go
import plotly.express as px

locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')
#read all csv files in this folder

#energiavertida = pd.read_excel('energiavertida.xlsx')
#energiavertida.index = energiavertida.din_instante
geracao = pd.read_csv('geracao.csv',sep=';',index_col='din_instante',parse_dates=True)
geracao = geracao[['id_estado','nom_usina','id_ons','val_geracao']]
geracao = geracao[geracao.id_estado=='PB']

intercambio = pd.read_csv('intercambio.csv',sep=';',index_col='din_instante',parse_dates=True)
intercambio = intercambio[['id_subsistema_origem','id_subsistema_destino','val_intercambiomwmed']]

linhas = pd.read_csv('linhas.csv',sep=';')

restr_eolica = pd.read_csv('restr_eolica.csv',sep=';',index_col='din_instante',parse_dates=True)
restr_eolica = restr_eolica[['id_estado','nom_usina','id_ons','val_geracao','val_geracaolimitada','val_disponibilidade','val_geracaoreferencia','val_geracaoreferenciafinal','cod_razaorestricao','cod_origemrestricao']]
restr_eolica = restr_eolica[restr_eolica.id_estado=='PB']
restr_fotovoltaica = pd.read_csv('restr_fotovoltaica.csv',sep=';',index_col='din_instante',parse_dates=True)
restr_fotovoltaica = restr_fotovoltaica[['id_estado','nom_usina','id_ons','val_geracao','val_geracaolimitada','val_disponibilidade','val_geracaoreferencia','val_geracaoreferenciafinal','cod_razaorestricao','cod_origemrestricao']]
restr_fotovoltaica = restr_fotovoltaica[restr_fotovoltaica.id_estado=='PB']
a=1

C:\Users\embra\AppData\Local\Temp\ipykernel_21596\339145024.py:24: DtypeWarning:

Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.



In [15]:
print('quantidade de geradores:',len(geracao.nom_usina.unique()))

quantidade de geradores: 17


Pegando dados de uma usina só como exemplo

In [72]:
restr_fotovoltaica.columns

Index(['id_estado', 'nom_usina', 'id_ons', 'val_geracao',
       'val_geracaolimitada', 'val_disponibilidade', 'val_geracaoreferencia',
       'val_geracaoreferenciafinal', 'cod_razaorestricao',
       'cod_origemrestricao'],
      dtype='object')

### Plot da geração e constrained off do Conj. Rio Alto (fotovoltaico)

In [ ]:
inicio = '2024-05-10 00:00:00'
fim = '2024-05-30 00:00:00'

rioalto = geracao[geracao.nom_usina=='Conj. Rio Alto']
rioalto = rioalto.loc[inicio:fim]
rioalto2 = restr_fotovoltaica[restr_fotovoltaica.nom_usina == 'CONJ. RIO ALTO']
rioalto2 = rioalto2.loc[inicio:fim]

In [71]:
fig = go.Figure()
#should be step type hv
fig.add_trace(go.Scatter(x=rioalto.index, y=rioalto.val_geracao, mode='lines',name='geração',line_shape='hv'))
fig.add_trace(go.Scatter(x=rioalto2.index, y=rioalto2.val_geracaolimitada, mode='lines',name='limite',line_shape='hv'))
fig.add_trace(go.Scatter(x=rioalto2.index, y=rioalto2.val_disponibilidade, mode='lines',name='disponibilidade',line_shape='hv'))
fig.add_trace(go.Scatter(x=rioalto2.index, y=rioalto2.val_geracaoreferencia, mode='lines',name='referencia',line_shape='hv'))
fig.add_trace(go.Scatter(x=rioalto2.index, y=rioalto2.val_geracaoreferenciafinal, mode='lines',name='referencia final',line_shape='hv'))


In [93]:
rioalto3 = rioalto2[rioalto2.val_geracaoreferencia > rioalto2.val_geracaolimitada]
rioalto3['perdida'] = rioalto3.val_geracaoreferencia - rioalto3.val_geracaolimitada
rioalto3.perdida.sum()/2


C:\Users\embra\AppData\Local\Temp\ipykernel_21596\3995553572.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



956.2399999999999

### Plot da geração e constrained off do  Conj. Luzia (eolico)

In [81]:
inicio = '2024-05-10 00:00:00'
fim = '2024-05-30 00:00:00'

chafariznorte = geracao[geracao.nom_usina=='Conj. Chafariz Norte']
chafariznorte = chafariznorte.loc[inicio:fim]
chafariznorte2 = restr_eolica[restr_eolica.nom_usina == 'CONJ. CHAFARIZ NORTE']
chafariznorte2 = chafariznorte2.loc[inicio:fim]

In [82]:
fig = go.Figure()
#should be step type hv
fig.add_trace(go.Scatter(x=chafariznorte.index, y=chafariznorte.val_geracao, mode='lines',name='geração',line_shape='hv'))
fig.add_trace(go.Scatter(x=chafariznorte2.index, y=chafariznorte2.val_geracaolimitada, mode='lines',name='limite',line_shape='hv'))
fig.add_trace(go.Scatter(x=chafariznorte2.index, y=chafariznorte2.val_disponibilidade, mode='lines',name='disponibilidade',line_shape='hv'))
fig.add_trace(go.Scatter(x=chafariznorte2.index, y=chafariznorte2.val_geracaoreferencia, mode='lines',name='referencia',line_shape='hv'))
fig.add_trace(go.Scatter(x=chafariznorte2.index, y=chafariznorte2.val_geracaoreferenciafinal, mode='lines',name='referencia final',line_shape='hv'))

In [94]:
chafariznorte3 = chafariznorte2[chafariznorte2.val_geracaoreferencia > chafariznorte2.val_geracaolimitada]
chafariznorte3['perdida'] = chafariznorte3.val_geracaoreferencia - chafariznorte3.val_geracaolimitada
chafariznorte3.perdida.sum()/2

C:\Users\embra\AppData\Local\Temp\ipykernel_21596\2974660589.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



17.4195

In [156]:
#delete all linhas that were deactivated, where linhas.dat_desativacao is not null
linhas = linhas[linhas.dat_desativacao.isnull()]
linhas

,id_subsistema_terminalde,nom_subsistema_terminalde,id_subsistema_terminalpara,nom_subsistema_terminalpara,id_estado_terminalde,nom_estado_de,id_estado_terminalpara,nom_estado_para,nom_subestacao_de,nom_subestacao_para,...,val_capacopercurtasemlimit,val_capacopercurtacomlimit,val_capacidadeoperveraodialonga,val_capacidadeoperveraonoitelonga,val_capacoperinvernodialonga,val_capacoperinvernonoitelonga,val_capacoperveradiacurta,val_capacoperveraonoitecurta,val_capacoperinvernodiacurta,val_capacoperinvernonoitecurta
0,SE,SUDESTE,SE,SUDESTE,AC,ACRE,AC,ACRE,FEIJO,RIO BRANCO I,...,1070.0,1070.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NE,NORDESTE,NE,NORDESTE,AL,ALAGOAS,AL,ALAGOAS,ARAPIRACA III,PENEDO,...,795.0,795.0,631.0,663.0,631.0,663.0,795.0,869.0,795.0,825.0
2,NE,NORDESTE,NE,NORDESTE,AL,ALAGOAS,AL,ALAGOAS,ARAPIRACA III,RIO LARGO II,...,795.0,795.0,631.0,663.0,631.0,663.0,795.0,870.0,795.0,824.0
3,NE,NORDESTE,NE,NORDESTE,AL,ALAGOAS,AL,ALAGOAS,MESSIAS,MACEIO,...,1346.0,1250.0,1068.0,1089.0,1068.0,1089.0,1346.0,1446.0,1349.0,1367.0
4,NE,NORDESTE,NE,NORDESTE,AL,ALAGOAS,AL,ALAGOAS,MESSIAS,MACEIO,...,1346.0,1250.0,1068.0,1089.0,1068.0,1089.0,1346.0,1446.0,1349.0,1367.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2255,SE,SUDESTE,SE,SUDESTE,SP,SAO PAULO,SP,SAO PAULO,ITABERA,TIJUCO PRETO,...,3930.0,3150.0,3119.0,3587.0,3311.0,3755.0,3930.0,4085.0,4113.0,4396.0
2256,N,NORTE,SE,SUDESTE,PA,PARA,MG,MINAS GERAIS,XINGU,ESTREITO,...,3500.0,3500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2257,N,NORTE,SE,SUDESTE,PA,PARA,MG,MINAS GERAIS,XINGU,ESTREITO,...,3500.0,3500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2258,N,NORTE,SE,SUDESTE,PA,PARA,RJ,RIO DE JANEIRO,XINGU,TERMINAL RIO,...,3500.0,3500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [166]:
#linhas.columns
linhaspbde = linhas[linhas.id_estado_terminalde == 'PB']
linhaspbpara = linhas[linhas.id_estado_terminalpara == 'PB']
linhaspb = linhaspb[['id_estado_terminalde','id_estado_terminalpara','nom_subestacao_de', 'nom_subestacao_para']]
len(linhaspbde.nom_linhadetransmissao.unique())


29

In [167]:
len(linhaspbpara.nom_linhadetransmissao.unique())

32

In [179]:
# list the items that are only in one of  
a = linhaspbde.nom_linhadetransmissao  
b = linhaspbpara.nom_linhadetransmissao
#list items that are in both a and b
[a.isin(b)]*

[593      True
 594      True
 595      True
 596      True
 605     False
 606     False
 607      True
 609     False
 611      True
 612      True
 613      True
 614      True
 615      True
 616      True
 617      True
 618      True
 619      True
 621      True
 622     False
 623     False
 1980    False
 1981    False
 1982    False
 1983     True
 1984    False
 1985     True
 1986     True
 1987    False
 1988     True
 Name: nom_linhadetransmissao, dtype: bool]

In [175]:
linhaspbde

,id_subsistema_terminalde,nom_subsistema_terminalde,id_subsistema_terminalpara,nom_subsistema_terminalpara,id_estado_terminalde,nom_estado_de,id_estado_terminalpara,nom_estado_para,nom_subestacao_de,nom_subestacao_para,...,val_capacopercurtasemlimit,val_capacopercurtacomlimit,val_capacidadeoperveraodialonga,val_capacidadeoperveraonoitelonga,val_capacoperinvernodialonga,val_capacoperinvernonoitelonga,val_capacoperveradiacurta,val_capacoperveraonoitecurta,val_capacoperinvernodiacurta,val_capacoperinvernonoitecurta
593,NE,NORDESTE,NE,NORDESTE,PB,PARAIBA,PB,PARAIBA,C.GRANDE II,C.GRANDE III,...,1590.0,1590.0,1265.0,1636.0,1309.0,1661.0,1590.0,1793.0,1590.0,1827.0
594,NE,NORDESTE,NE,NORDESTE,PB,PARAIBA,PB,PARAIBA,C.GRANDE II,C.GRANDE III,...,1590.0,1590.0,1265.0,1636.0,1309.0,1661.0,1590.0,1793.0,1590.0,1827.0
595,NE,NORDESTE,NE,NORDESTE,PB,PARAIBA,PB,PARAIBA,C.GRANDE II,C.GRANDE III,...,1590.0,1590.0,1265.0,1265.0,1265.0,1265.0,1590.0,1590.0,1590.0,1590.0
596,NE,NORDESTE,NE,NORDESTE,PB,PARAIBA,PB,PARAIBA,C.GRANDE II,COTEMINAS,...,1590.0,1200.0,1262.0,1325.0,1262.0,1325.0,1590.0,1751.0,1613.0,1650.0
605,NE,NORDESTE,NE,NORDESTE,PB,PARAIBA,RN,RIO GRANDE DO NORTE,C.GRANDE II,PARAISO,...,795.0,795.0,631.0,663.0,631.0,663.0,795.0,863.0,795.0,809.0
606,NE,NORDESTE,NE,NORDESTE,PB,PARAIBA,RN,RIO GRANDE DO NORTE,C.GRANDE II,PARAISO,...,795.0,795.0,631.0,663.0,631.0,663.0,795.0,863.0,795.0,809.0
607,NE,NORDESTE,NE,NORDESTE,PB,PARAIBA,PB,PARAIBA,C.GRANDE II,UT.CAM.GRANDE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
609,NE,NORDESTE,NE,NORDESTE,PB,PARAIBA,RN,RIO GRANDE DO NORTE,C.GRANDE III,EXTREMOZ II,...,1590.0,1590.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
611,NE,NORDESTE,NE,NORDESTE,PB,PARAIBA,PB,PARAIBA,COREMAS,RIO ALTO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
612,NE,NORDESTE,NE,NORDESTE,PB,PARAIBA,PB,PARAIBA,EPASA,MUSSURE II,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [144]:
len(linhaspbde.nom_subestacao_para.unique())
#linhaspbde

18

In [151]:
len(linhaspbde)

29

In [128]:
# linhas[linhas.id_estado_terminalde == 'PB'] or linhas[linhas.id_estado_terminalpara == 'PB']
#linhas[(linhas.id_estado_terminalde == 'PB') | (linhas.id_estado_terminalpara == 'PB')]